In [20]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
from tqdm import tqdm

In [2]:
results:dict = json.load(open("/home/pavel/etalab/code/csv_detective_api/csv_data_rules.json"))

In [3]:
results["344a19ea-0eec-46e8-aaeb-6f8d53ade88b"]

{'encoding': 'UTF-8',
 'separator': ',',
 'header_row_idx': 0,
 'header': ['identifiant',
  'type_de_structure',
  'libelle_type_de_structure',
  'libelle_court',
  'libelle_long',
  'siege_administratif',
  'geolocalisation',
  'latitude',
  'longitude',
  'adresse',
  'complement_adresse',
  'code_postal',
  'localite',
  'site_internet',
  'compte_twitter',
  'siret',
  'element_wikidata',
  'compte_facebook',
  'compte_linkedin',
  'compte_viadeo',
  'compte_dailymotion',
  'compte_youtube',
  'code_commune',
  'commune',
  'code_unite_urbaine',
  'unite_urbaine',
  'code_departement',
  'departement',
  'code_academie',
  'academie',
  'code_region',
  'region'],
 'total_lines': 18,
 'heading_columns': 0,
 'trailing_columns': 0,
 'ints_as_floats': [],
 'columns_rb': {'siege_administratif': ['booleen'],
  'geolocalisation': ['latlon_wgs'],
  'latitude': ['latitude_wgs',
   'longitude_wgs',
   'longitude_wgs_fr_metropole',
   'longitude_l93'],
  'longitude': ['latitude_wgs',
   'lon

In [4]:
# Find the distinct types found by the ML method

all_types = set([])

for id_, metadata in results.items():
    if "columns_rb" in metadata and metadata["columns_rb"]:
        for col_name, type_ in metadata["columns_rb"].items():
            all_types.update(type_)
        
"""
{'adresse',
 'booleen',
 'code_commune_insee',
 'code_csp_insee',
 'code_departement',
 'code_fantoir',
 'code_postal',
 'code_region',
 'commune',
 'csp_insee',
 'date',
 'date_fr',
 'datetime_iso',
 'departement',
 'email',
 'insee_ape700',
 'insee_canton',
 'iso_country_code',
 'jour_de_la_semaine',
 'json_geojson',
 'latitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'latlon_wgs',
 'longitude_l93',
 'longitude_wgs',
 'longitude_wgs_fr_metropole',
 'pays',
 'region',
 'sexe',
 'siren',
 'siret',
 'tel_fr',
 'twitter',
 'uai',
 'url',
 'year'}
 """

"\n{'adresse',\n 'booleen',\n 'code_commune_insee',\n 'code_csp_insee',\n 'code_departement',\n 'code_fantoir',\n 'code_postal',\n 'code_region',\n 'commune',\n 'csp_insee',\n 'date',\n 'date_fr',\n 'datetime_iso',\n 'departement',\n 'email',\n 'insee_ape700',\n 'insee_canton',\n 'iso_country_code',\n 'jour_de_la_semaine',\n 'json_geojson',\n 'latitude_l93',\n 'latitude_wgs',\n 'latitude_wgs_fr_metropole',\n 'latlon_wgs',\n 'longitude_l93',\n 'longitude_wgs',\n 'longitude_wgs_fr_metropole',\n 'pays',\n 'region',\n 'sexe',\n 'siren',\n 'siret',\n 'tel_fr',\n 'twitter',\n 'uai',\n 'url',\n 'year'}\n "

In [43]:
# Types not to annotate (by me)
non_annotate_types = {
 'code_csp_insee',
 'code_fantoir',
 'csp_insee',
 'insee_ape700',
 'insee_canton',
 'latitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'latlon_wgs',
 'longitude_l93',
 'longitude_wgs',
 'longitude_wgs_fr_metropole',
 'uai',
}



In [5]:
# Create dict with iinverted index {type: (col_name, id_)}
iindex_types_cols = defaultdict(list)
for id_, metadata in results.items():
    if "columns_rb" in metadata and metadata["columns_rb"]:
        for col_name, type_ in metadata["columns_rb"].items():
            iindex_types_cols[type_[0]].append((col_name, id_))

In [18]:
def get_sample(results, id_, header, n_sample=10):
    encoding = "utf-8"
    separator = ","
    if id_ in results:
        if "encoding" in results[id_]:
            encoding = results[id_]["encoding"]
        if "separator" in results[id_]:
            separator = results[id_]["separator"]
    try:        
        df = pd.read_csv(f"/data/datagouv/datagouv_full/{id_}.csv", sep=separator, encoding=encoding)
    except:
        print(f"Could not read {id_} file")
        return []
    header = header.replace('"', "")
    if header in df.columns:
        n_sample = min(len(df[header].dropna()), n_sample)
        sampled_values = df[header].dropna().sample(n=n_sample, random_state=42).values
        return sampled_values
    else:
        print(f"Could not find column {header} in file {id_}")
        return []
    
    
    
    
            
        

In [7]:
iindex_types_cols["siren"]

[('EPCI 2016', 'c7d9fa8f-fbea-40d7-8090-e6226910439c'),
 ('FINESS géographique', '0bf69358-fca8-496b-961e-1dc2faeba89c'),
 ('finess', '3a1987ed-a4f8-4da9-a167-4ae01272a768'),
 ('finessEJ', '3a1987ed-a4f8-4da9-a167-4ae01272a768'),
 ('Code EPCI', '69e3f577-0384-4488-9305-5f05534b0dd1'),
 ('siren', '5bbd76e7-6966-47c4-9f32-9a678b963bdc'),
 ('finess', '0e5bb417-18b1-48d4-886c-f796b6b39f6b'),
 ('ref:FR:FINESS', 'db19d4e9-779e-4f58-8e66-620a5c1aa958'),
 ('finess', '84da3870-266c-4675-94dd-3ee037d668f5'),
 ('siren', '65838a1b-7c6e-4efa-9877-a0b9362e0ced'),
 ('finess', 'f8cf741f-1a4c-43b2-8b00-a9025db9cf53'),
 ('siren_epci', '678ca756-0d8a-45a2-bf3c-57f3ccb47b78'),
 ('nofinesset', '6d03f6e7-0936-4328-ae11-c266be8944b5'),
 ('nofinessej', '6d03f6e7-0936-4328-ae11-c266be8944b5'),
 ('SIREN', '46acb713-6d22-46b3-adcc-10cb93ce1c7e'),
 ('FINESS_ET', 'd0075329-2e20-4b0b-a3e9-f8c182e8d815'),
 ('FINESS_EJ', 'd0075329-2e20-4b0b-a3e9-f8c182e8d815'),
 ('SIREN', '6b47c253-81a8-4464-ad69-cd78c7a157a6'),
 ('f

In [30]:
list_rows = []
n_sampled_files = 100
np.random.seed(42) 
for type_, val_tups in tqdm(iindex_types_cols.items()):
    sample = np.random.choice(len(val_tups), min(n_sampled_files, len(val_tups)),
                              replace=False)
    sample = np.array(val_tups)[sample]
    row_dict = {}
    for col_name, id_ in tqdm(sample):
        row_dict = {}
        row_dict["columns"] = col_name
        row_dict["sample"] = get_sample(results, id_=id_, header=col_name, n_sample=10)
        row_dict["human_detected"] = type_
        row_dict["csv_detected"] = type_
        row_dict["id"] = id_
        list_rows.append(dict(row_dict))
        
df_rule_annotated = pd.DataFrame(list_rows)
df_rule_annotated = df_rule_annotated[["columns", "sample", "human_detected", "csv_detected", "id"]]
df_rule_annotated.to_csv("../csv_detective_ml/data/distant_supervised_all.csv", index=False)



  0%|          | 0/34 [00:00<?, ?it/s]


  0%|          | 0/100 [00:00<?, ?it/s]/home/pavel/anaconda3/envs/csv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (67,94) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):



  2%|▏         | 2/100 [00:01<00:56,  1.72it/s]


  4%|▍         | 4/100 [00:01<00:40,  2.36it/s]


  6%|▌         | 6/100 [00:01<00:32,  2.88it/s]


  7%|▋         | 7/100 [00:02<00:33,  2.75it/s]


 11%|█         | 11/100 [00:02<00:24,  3.68it/s]


 17%|█▋        | 17/100 [00:02<00:16,  5.10it/s]


 20%|██        | 20/100 [00:02<00:12,  6.36it/s]


 22%|██▏       | 22/100 [00:02<00:12,  6.10it/s]


 28%|██▊       | 28/100 [00:03<00:11,  6.12it/s]


 31%|███       | 31/100 [00:04<00:08,  7.74it/s]


 35%|███▌      | 35/100 [00:05<00:12,  5.31it/s]


 38%|███▊      | 38/100 [00:07<00:21,  2.89it/s]


 42%|████▏     | 42/100 [00:08<00:18,  3.16it/s]


 47%|██




 65%|██████▌   | 65/100 [01:05<01:46,  3.04s/it]


 71%|███████   | 71/100 [01:06<01:01,  2.13s/it]


 74%|███████▍  | 74/100 [01:06<00:39,  1.51s/it]


 76%|███████▌  | 76/100 [01:06<00:26,  1.10s/it]


 78%|███████▊  | 78/100 [01:06<00:18,  1.21it/s]


 80%|████████  | 80/100 [01:15<00:37,  1.86s/it]


 82%|████████▏ | 82/100 [01:15<00:23,  1.32s/it]


 86%|████████▌ | 86/100 [01:15<00:13,  1.07it/s]


 94%|█████████▍| 94/100 [01:16<00:04,  1.47it/s]


 96%|█████████▌| 96/100 [01:18<00:03,  1.21it/s]


 98%|█████████▊| 98/100 [01:18<00:01,  1.68it/s]


100%|██████████| 100/100 [01:18<00:00,  2.28it/s]

  9%|▉         | 3/34 [03:13<34:33, 66.90s/it]


  0%|          | 0/100 [00:00<?, ?it/s]


  7%|▋         | 7/100 [00:00<00:01, 57.41it/s]


  7%|▋         | 7/100 [00:10<00:01, 57.41it/s]


 12%|█▏        | 12/100 [00:10<00:56,  1.57it/s]


 19%|█▉        | 19/100 [00:10<00:36,  2.22it/s]


 25%|██▌       | 25/100 [00:10<00:24,  3.12it/s]


 33%|███▎      | 33/100 [00:13<00:21,  3.

 52%|█████▏    | 52/100 [00:21<00:19,  2.47it/s]


 60%|██████    | 60/100 [00:21<00:11,  3.48it/s]


 64%|██████▍   | 64/100 [00:29<00:27,  1.29it/s]


 71%|███████   | 71/100 [00:29<00:16,  1.79it/s]


 77%|███████▋  | 77/100 [00:29<00:09,  2.53it/s]


 82%|████████▏ | 82/100 [00:29<00:05,  3.53it/s]


 86%|████████▌ | 86/100 [00:29<00:02,  4.75it/s]


 91%|█████████ | 91/100 [00:30<00:01,  6.41it/s]


 96%|█████████▌| 96/100 [00:30<00:00,  8.59it/s]


100%|██████████| 100/100 [00:30<00:00,  3.31it/s]

 18%|█▊        | 6/34 [06:34<30:43, 65.82s/it]


  0%|          | 0/100 [00:00<?, ?it/s]


  5%|▌         | 5/100 [00:00<00:02, 44.70it/s]


  8%|▊         | 8/100 [00:00<00:03, 26.31it/s]


 10%|█         | 10/100 [00:00<00:05, 17.27it/s]


 12%|█▏        | 12/100 [00:13<02:51,  1.95s/it]


 13%|█▎        | 13/100 [00:26<07:37,  5.26s/it]


 15%|█▌        | 15/100 [00:26<05:14,  3.70s/it]


 18%|█▊        | 18/100 [00:26<03:33,  2.60s/it]


 20%|██        | 20/100 [00:26<02:27,  1.84s




 16%|█▌        | 16/100 [00:09<01:04,  1.30it/s]/home/pavel/anaconda3/envs/csv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (41,76,181,196) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):



 18%|█▊        | 18/100 [00:09<00:46,  1.78it/s]


 22%|██▏       | 22/100 [00:10<00:34,  2.25it/s]


 24%|██▍       | 24/100 [00:10<00:25,  3.04it/s]


 25%|██▌       | 25/100 [00:13<01:16,  1.01s/it]


 26%|██▌       | 26/100 [00:13<01:00,  1.22it/s]


 28%|██▊       | 28/100 [00:15<01:02,  1.16it/s]


 30%|███       | 30/100 [00:16<00:46,  1.50it/s]


 32%|███▏      | 32/100 [00:16<00:34,  1.97it/s]


 36%|███▌      | 36/100 [00:16<00:23,  2.76it/s]


 38%|███▊      | 38/100 [00:16<00:18,  3.32it/s]


 41%|████      | 41/100 [00:17<00:14,  3.94it/s]


 43%|████▎     | 43/100 [00:17<00:12,  4.62it/s]


 47%|████▋     | 47/100 [00:17<00:09,  5.62it/s]


 49%|████▉     | 49/100 [0

 87%|████████▋ | 87/100 [00:34<00:04,  2.71it/s]


 89%|████████▉ | 89/100 [00:34<00:03,  3.62it/s]


 91%|█████████ | 91/100 [00:35<00:02,  3.45it/s]


 97%|█████████▋| 97/100 [00:35<00:00,  4.79it/s]


100%|██████████| 100/100 [00:35<00:00,  2.84it/s]

 35%|███▌      | 12/34 [09:51<13:43, 37.43s/it]


  0%|          | 0/100 [00:00<?, ?it/s]


  2%|▏         | 2/100 [00:00<00:15,  6.19it/s]


  5%|▌         | 5/100 [00:00<00:11,  8.00it/s]


  6%|▌         | 6/100 [00:00<00:11,  8.03it/s]


  8%|▊         | 8/100 [00:00<00:10,  8.42it/s]


  9%|▉         | 9/100 [00:00<00:10,  8.60it/s]


 12%|█▏        | 12/100 [00:01<00:08, 10.22it/s]


 14%|█▍        | 14/100 [00:01<00:07, 11.30it/s]


 16%|█▌        | 16/100 [00:01<00:07, 10.69it/s]


 18%|█▊        | 18/100 [00:01<00:08, 10.10it/s]


 20%|██        | 20/100 [00:01<00:07, 10.33it/s]


 24%|██▍       | 24/100 [00:01<00:06, 12.52it/s]


 26%|██▌       | 26/100 [00:12<02:03,  1.67s/it]


 28%|██▊       | 28/100 [00:12<01:25,  1.19s/i

 25%|██▌       | 25/100 [00:03<00:21,  3.43it/s]


 27%|██▋       | 27/100 [00:04<00:21,  3.33it/s]


 33%|███▎      | 33/100 [00:04<00:14,  4.52it/s]


 36%|███▌      | 36/100 [00:04<00:10,  5.96it/s]


 42%|████▏     | 42/100 [00:04<00:07,  8.13it/s]


 45%|████▌     | 45/100 [00:04<00:05, 10.16it/s]


 48%|████▊     | 48/100 [00:05<00:04, 11.06it/s]


 51%|█████     | 51/100 [00:05<00:05,  8.96it/s]


 53%|█████▎    | 53/100 [00:17<01:26,  1.84s/it]


 54%|█████▍    | 54/100 [00:17<01:01,  1.33s/it]


 58%|█████▊    | 58/100 [00:17<00:39,  1.06it/s]


 63%|██████▎   | 63/100 [00:17<00:24,  1.50it/s]


 68%|██████▊   | 68/100 [00:17<00:15,  2.11it/s]


 72%|███████▏  | 72/100 [00:18<00:09,  2.85it/s]


 76%|███████▌  | 76/100 [00:18<00:06,  3.95it/s]


 80%|████████  | 80/100 [00:18<00:03,  5.39it/s]


 85%|████████▌ | 85/100 [00:19<00:02,  5.29it/s]


 88%|████████▊ | 88/100 [00:19<00:01,  6.70it/s]


 91%|█████████ | 91/100 [00:30<00:10,  1.20s/it]


 97%|█████████▋| 97/100 [00:30<

 64%|██████▍   | 64/100 [00:02<00:01, 25.27it/s]/home/pavel/anaconda3/envs/csv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):



 67%|██████▋   | 67/100 [00:04<00:06,  5.41it/s]


 71%|███████   | 71/100 [00:04<00:04,  7.19it/s]


 74%|███████▍  | 74/100 [00:04<00:02,  9.13it/s]


 81%|████████  | 81/100 [00:04<00:01, 12.30it/s]


 86%|████████▌ | 86/100 [00:05<00:01, 11.40it/s]


 92%|█████████▏| 92/100 [00:05<00:00, 14.87it/s]


 96%|█████████▌| 96/100 [00:05<00:00, 17.78it/s]


100%|██████████| 100/100 [00:05<00:00, 18.77it/s]

 59%|█████▉    | 20/34 [13:14<03:51, 16.52s/it]


  0%|          | 0/100 [00:00<?, ?it/s]/home/pavel/anaconda3/envs/csv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if

 53%|█████▎    | 53/100 [01:54<01:53,  2.42s/it]


 55%|█████▌    | 55/100 [01:57<01:34,  2.10s/it]


 56%|█████▌    | 56/100 [01:57<01:06,  1.51s/it]


 60%|██████    | 60/100 [01:57<00:43,  1.09s/it]


 62%|██████▏   | 62/100 [01:57<00:29,  1.28it/s]


 63%|██████▎   | 63/100 [02:00<00:54,  1.47s/it]/home/pavel/anaconda3/envs/csv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (1,15,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):



 64%|██████▍   | 64/100 [02:23<04:44,  7.90s/it]


 65%|██████▌   | 65/100 [02:23<03:14,  5.56s/it]


 68%|██████▊   | 68/100 [02:23<02:05,  3.91s/it]


 71%|███████   | 71/100 [02:24<01:20,  2.76s/it]


 75%|███████▌  | 75/100 [02:24<00:48,  1.94s/it]


 79%|███████▉  | 79/100 [02:24<00:28,  1.37s/it]


 82%|████████▏ | 82/100 [02:32<00:31,  1.78s/it]


 85%|████████▌ | 85/100 [02:32<00:18,  1.26s/it]


 87%|████████▋ | 87/100 [02:36<00:1

 46%|████▌     | 46/100 [03:39<09:39, 10.73s/it]


 50%|█████     | 50/100 [04:03<07:45,  9.31s/it]


 51%|█████     | 51/100 [04:03<05:21,  6.55s/it]


 52%|█████▏    | 52/100 [04:17<07:00,  8.77s/it]


 54%|█████▍    | 54/100 [04:23<05:25,  7.08s/it]


 55%|█████▌    | 55/100 [04:25<04:13,  5.64s/it]


 56%|█████▌    | 56/100 [04:48<07:56, 10.83s/it]


 59%|█████▉    | 59/100 [04:58<05:50,  8.55s/it]


 60%|██████    | 60/100 [05:00<04:21,  6.54s/it]


 61%|██████    | 61/100 [05:02<03:24,  5.24s/it]


 62%|██████▏   | 62/100 [05:10<03:56,  6.22s/it]


 65%|██████▌   | 65/100 [05:12<02:38,  4.54s/it]


 67%|██████▋   | 67/100 [05:26<02:54,  5.28s/it]


 68%|██████▊   | 68/100 [05:29<02:26,  4.57s/it]


 69%|██████▉   | 69/100 [05:52<05:12, 10.08s/it]


 70%|███████   | 70/100 [05:55<03:53,  7.79s/it]


 72%|███████▏  | 72/100 [05:57<02:42,  5.82s/it]


 73%|███████▎  | 73/100 [06:00<02:13,  4.95s/it]


 74%|███████▍  | 74/100 [06:13<03:14,  7.47s/it]


 75%|███████▌  | 75/100 [06:15<

In [25]:
list_rows[0]
df_rule_annotated = pd.DataFrame(list_rows)

In [27]:
df_rule_annotated = df_rule_annotated[["columns", "sample", "human_detected", "csv_detected", "id"]]

In [29]:
df_rule_annotated.to_csv("../csv_detective_ml/data/distant_supervised_all.csv", index=False)